<a href="https://colab.research.google.com/github/chetools/CHE4071_Fall2024/blob/main/DynamicDistillation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from scipy.integrate import solve_ivp
import numpy as np
from plotly.subplots import make_subplots

In [12]:
#0  Condenser
#1
#2  Nr-1
#3  Feed Stage
#4
#5
#6  Nr+Ns-1
#7  Nr+Ns:  Reboiler

Nr = 3
Ns = 4

F=1.5  #mol/s
z=0.4
q = 1.
R=2.
D=F/2
alpha=2.
stage_holdup=0.5
feedstage_holdup = 0.5
condenser_holdup = 1.
reboiler_holdup = 1.
m_holdup=np.r_[condenser_holdup, np.full(Nr-1, stage_holdup), feedstage_holdup, np.full(Ns-1, stage_holdup) ,
               reboiler_holdup]
W=1.  # 1/(s mol**0.5)

#initial conditions
m0 = m_holdup*1.05  #initial number moles on each stage is set slightly higher than holdup to yield a little flow
m0[0] = (((R+1)*D)/W)**(2/3)+condenser_holdup
x0 = np.full_like(m0, z)  #fill each stage with feed composition


In [13]:
def ramp(t, a, b):
    if t<a:
        return 0.
    if t>b:
        return 1.
    return (t-a)/(b-a)


In [14]:
fig=make_subplots()
tplot = np.linspace(0,100)
yplot = [2*ramp(t,25,75)+1 for t in tplot]

fig.add_scatter(x=tplot, y=yplot)

In [50]:
def rhs(t, v, reflux_ratio_function):
    R=reflux_ratio_function(t)
    D=ramp(t, 50,100)*F/2
    m,x = np.split(v,2)
    y=alpha*x/(1-x+alpha*x)
    dm = np.zeros_like(m)
    dmx = np.zeros_like(m)
    V = np.zeros_like(m)

    L =  np.where(m<m_holdup, 0., W*(m-m_holdup)**1.5)

    L[0]=R*D

    Vrec = L[0] + D

    V[1:Nr+1] = Vrec  #constant vapor flow rate in rectifying section
    V[Nr+1:] = Vrec - F*(1-q)  #constant vapor flow rate in stripping section

    dm[1:-1] =  L[:-2] - L[1:-1] + V[2:] - V[1:-1]
    #  1to6      0to5      1to6     2to7     1to6
    dm[Nr]+=F
    dm[0] = V[1] - L[0] - D
    dm[-1] = L[-2] - V[-1] - L[-1]

    dmx[1:-1] =  x[:-2]*L[:-2] - x[1:-1]*L[1:-1] + y[2:]*V[2:] - y[1:-1]*V[1:-1]
    dmx[Nr]+=z*F
    dmx[0] = y[1]*V[1] - x[0]*L[0] - x[0]*D
    dmx[-1] = x[-2]*L[-2] - y[-1]*V[-1] - x[-1]*L[-1]

    dx = (dmx - x*dm)/m
    return np.r_[dm, dx]

In [51]:
tend=300
res1=solve_ivp(rhs, (0, tend), np.r_[m0, x0], method='Radau', dense_output=True, args=(lambda t: 2.,))
res1

<ipython-input-50-fa23d472e0b9>:10: RuntimeWarning:

invalid value encountered in power



  message: The solver successfully reached the end of the integration interval.
  success: True
   status: 0
        t: [ 0.000e+00  6.120e-02 ...  2.443e+02  3.000e+02]
        y: [[ 2.717e+00  2.717e+00 ...  2.717e+00  2.717e+00]
            [ 5.250e-01  5.248e-01 ...  1.810e+00  1.810e+00]
            ...
            [ 4.000e-01  4.000e-01 ...  1.458e-01  1.458e-01]
            [ 4.000e-01  4.000e-01 ...  8.965e-02  8.965e-02]]
      sol: <scipy.integrate._ivp.common.OdeSolution object at 0x7e4d628f63e0>
 t_events: None
 y_events: None
     nfev: 398
     njev: 18
      nlu: 94

In [52]:
tplot=np.linspace(0,tend,100)
fig = make_subplots(rows=1, cols=2)
ms,xs = np.split(res1.sol(tplot),2,axis=0)
for i,m in enumerate(ms):
    fig.add_scatter(x=tplot, y=m, row=1,col=1, name=f'moles {i}')

for i,x in enumerate(xs):
    fig.add_scatter(x=tplot, y=x, row=1,col=2, name=f'molfrac {i}')

fig.update_layout(width=800, height=400, template='plotly_dark')



In [53]:
xplot=np.linspace(0,1,100)
xss=xs[:,-1]
yss= y=alpha*xss/(1-xss+alpha*xss)
xsteps = np.r_[xss[0], np.repeat(xss[1:-1],2), xss[-1] ]
ysteps=np.repeat(yss[1:],2)

fig=make_subplots()
fig.add_scatter(x=[0,1],y=[0,1],mode='lines',line_color='grey')
fig.add_scatter(x=xplot, y=alpha*xplot/(1-xplot+alpha*xplot), line_color='green')
fig.add_scatter(x=xsteps, y=ysteps, mode='lines', line_color='red')
fig.update_layout(width=500,height=500, template='plotly_dark', showlegend=False)

In [78]:
tend=300
res2=solve_ivp(rhs, (0, tend), res1.y[:,-1], method='Radau', dense_output=True, args=(lambda t: 3*ramp(t, 10, 100)+2,))
res2

<ipython-input-50-fa23d472e0b9>:10: RuntimeWarning:

invalid value encountered in power



  message: The solver successfully reached the end of the integration interval.
  success: True
   status: 0
        t: [ 0.000e+00  6.011e-02 ...  2.295e+02  3.000e+02]
        y: [[ 2.717e+00  2.717e+00 ...  2.717e+00  2.717e+00]
            [ 1.810e+00  1.725e+00 ...  2.914e+00  2.914e+00]
            ...
            [ 1.458e-01  1.506e-01 ...  9.301e-02  9.291e-02]
            [ 8.965e-02  9.498e-02 ...  5.250e-02  5.244e-02]]
      sol: <scipy.integrate._ivp.common.OdeSolution object at 0x7e4d5e12f310>
 t_events: None
 y_events: None
     nfev: 467
     njev: 19
      nlu: 98

In [81]:
tplot=np.linspace(0,tend,5)
mxs = res2.sol(tplot)

fig=make_subplots(rows=1, cols=5, subplot_titles=[f'time = {t:.0f}' for t in tplot])
xplot=np.linspace(0,1,100)


for i in range(tplot.size):
    ms, xs = np.split(mxs[:,i],2)
    ys= y=alpha*xs/(1-xs+alpha*xs)
    xsteps = np.r_[xs[0], np.repeat(xs[1:-1],2), xs[-1] ]
    ysteps=np.repeat(ys[1:],2)

    fig.add_scatter(x=[0,1],y=[0,1],mode='lines',line_color='grey',row=1,col=i+1)
    fig.add_scatter(x=xplot, y=alpha*xplot/(1-xplot+alpha*xplot), line_color='green', row=1,col=i+1)
    fig.add_scatter(x=xsteps, y=ysteps, mode='lines', line_color='red', row=1,col=i+1)
fig.update_layout(width=1200,height=400, template='plotly_dark', showlegend=False)